In [ ]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint
import matplotlib

from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

from src.UVMappingPipeline import UVMappingPipeline
from utils.sys_utils import *


pipe = UVMappingPipeline()

In [ ]:
%reload_ext autoreload
plt.close('all')
tdir = "/home/hunter/data/images/obstacles"
# tdir = homeDir
found_imgs, _ = grab_dir_images(tdir)


p = interactive(pipe.pipeline,
    _img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ),
    line_method = {'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    input_method = {'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1 = (1,255,1), e2 = (1,255,1),
    ang = (0,360,1), rho = (1,1000,1),
    minLineLength = (0,255,1), maxLineGap = (0,255,1),
    houghThresh = (0,255,1), greyThreshU = (0,255,1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False
)


row1 = HBox([p.children[0]])
row2 = HBox([p.children[1], p.children[2]], layout = Layout(flex_flow='row wrap'))
row3 = HBox([p.children[3], p.children[4]], layout = Layout(flex_flow='row wrap'))
row4 = HBox([p.children[5], p.children[6], p.children[7], p.children[8]], layout = Layout(display='flex',flex_flow='row'))
row5 = HBox([p.children[9], p.children[10], p.children[11]], layout = Layout(flex_flow='row wrap'))
row6 = HBox([p.children[12], p.children[13], p.children[14]], layout = Layout(flex_flow='row wrap'))
controls = VBox([row1,row2,row3,row4,row5,row6], layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

In [ ]:
print(pipe.nObs)
idx = 0

plt.figure(8)
plt.imshow(pipe.obsImgs[idx])
plt.show()

plt.figure(2)
plt.imshow(pipe.display_obstacles)
plt.show()

In [ ]:
# print(pipe.disparityBounds[obsId],pipe.xBounds[obsId])

tmp = np.copy(pipe.greyV)
tmp2 = np.copy(pipe.img)
tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
nObs = len(pipe.disparityBounds)
print(nObs)
b = 2; d = np.ceil(nObs / float(b))

plt.close(7)
fig, axes = plt.subplots(nrows=b, ncols=int(d), num=7)
obstacleImgs = []
for i in range(nObs):
    tmp2 = np.copy(pipe.img)
    tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
    try:
        xs = pipe.xBounds[i]
        ys,disp = pipe.find_obstacles(tmp,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True)
        cv2.rectangle(tmp2,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
    except:
        pass
    fig.axes[i].imshow(disp)
    obstacleImgs.append(tmp2)
plt.show()

In [ ]:
%reload_ext autoreload
# plt.figure(7)
h,w = pipe.vmap.shape[:2]
vmask = np.zeros((h,w,3), dtype=np.uint8)
cv2.rectangle(vmask,(0,0),(5,h),(255,255,255), cv2.FILLED)

mask = cv2.cvtColor(vmask,cv2.COLOR_BGR2GRAY)
mask_inv = cv2.bitwise_not(mask)

tmpV = np.copy(pipe.vmap)
tmpV = cv2.cvtColor(tmpV,cv2.COLOR_GRAY2BGR)
_, greyV = cv2.threshold(tmpV,11,255,cv2.THRESH_BINARY)

maskV, invMaskV = pipe.get_vmap_mask(greyV,verbose=True)

VMASK = cv2.bitwise_or(maskV, mask)
cv2.rectangle(invMaskV,(0,0),(5,h),(0,0,0), cv2.FILLED)

res = cv2.bitwise_and(pipe.vmap, pipe.vmap, mask = invMaskV)



_, greyV = cv2.threshold(res,11,255,cv2.THRESH_BINARY)

# plt.imshow(greyV)

cntsV, hierV = pipe.find_contours(greyV,20.0)
pipe.contoursV = cntsV; pipe.contour_hierarchyV = hierV

_cnts, limits, ellipses, pts, centers = pipe.extract_contour_x_bounds(cntsV)
# x,y,w,h = cv2.boundingRect(cntsV[-3])
# print(x, x+w, y, y+h)
# cv2.rectangle(greyV,(x,y),(x+w,y+h),(150,0,0),1)

pipe.plot_contours(greyV,cntsV,ellipses,centers,pts)

# plt.show()

In [ ]:
img = np.copy(pipe.img)
sobelx8u = cv2.Sobel(img,cv2.CV_8U,1,0,ksize=3)

# Output dtype = cv2.CV_64F. Then take its absolute and convert to cv2.CV_8U
sobelx64f = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=3)
abs_sobel64f = np.absolute(sobelx64f)
sobel_8u = np.uint8(abs_sobel64f)

plt.subplot(1,3,1),plt.imshow(img,cmap = 'gray')
plt.title('Original'), plt.xticks([]), plt.yticks([])
plt.subplot(1,3,2),plt.imshow(sobelx8u,cmap = 'gray')
plt.title('Sobel CV_8U'), plt.xticks([]), plt.yticks([])
plt.subplot(1,3,3),plt.imshow(sobel_8u,cmap = 'gray')
plt.title('Sobel abs(CV_64F)'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(pipe.dilatedU,kernel,iterations = 1)

comp_imgs = [pipe.umap,pipe.greyU,pipe.dilatedU,pipe.blurredU,pipe.closingU,erosion]
composite = pipe.construct_composite_img(comp_imgs,2,3)
pipe.plot_image(composite,3)

In [ ]:
posU = []
posV = []
def onclick(event):
    if event.dblclick:
        posU.append([event.xdata,event.ydata])
        print(posU)
#     print(event.ydata)

def onclickV(event):
    if event.dblclick:
        posV.append([event.xdata,event.ydata])
        print(posV)

In [ ]:
posU = []
posV = []
fu = plt.figure(6)
fu.canvas.mpl_connect('button_press_event', onclick)
plt.imshow(pipe.umap)
plt.show()

# fv = plt.figure(7)
# fv.canvas.mpl_connect('button_press_event', onclickV)

In [ ]:
plt.figure(7)
plt.imshow(pipe.vmap)
plt.show()

In [ ]:
plt.figure(6)
tmp = np.copy(pipe.img)

# wx_low = 13
# wy_high = 295
# wx_high = 130
# wy_low = 40

wx_low = 130
wy_high = 323
wx_high = 423
wy_low = 164


cv2.rectangle(tmp,(wx_low,wy_high),(wx_high,wy_low),(255,255,0), 1)
cv2.rectangle(tmp,(543,323),(640,164),(255,255,0), 1)

plt.imshow(tmp)
plt.show()

In [ ]:
# Image (frame_168 w/ 400 threshold disparities b/w 140 and 170)
cnts2, limits, ellipses, pts, centers = pipe.extract_contour_x_bounds()
print(limits)
# pp.pprint(hier)

temp = np.copy(pipe.img)
print(temp.shape)

goodxs = np.uint8(range(75,100))
goodpixels = np.uint8(range(140,425))

ix = np.isin(temp, goodxs)
ipx = np.isin(temp, goodpixels)
print(ix.shape,ipx.shape)

a = np.where(ix)
# b = np.isin(ix[1],range(140,425))
# c = np.where(b)

for i in range(len(a[0])):
    y = a[0][i]
    x = a[1][i]
    if(x>=140 and x <= 425):
#         print(x,y)
        temp[y,x] = (124)
    

print(a[1].shape)
# print(np.max(c[1]))
# print(np.max(a[1]))
# temp[c] = (123)

# print(a)

plt.figure(0)
plt.imshow(temp)
plt.show()

In [ ]:
print("""
Good Configuration:    for image = test/test_disparity.png)
---------------------------------------------------------------
    
    * Mapping Space       : U-Map
    * Line Finding Method : Probablistic Hough Transform
    * Filtering Used      : Composite Filtering -> Blurring
    * Kernel Size         : (2, 3)
    * Rho, Angle (deg)    :  1, 90
    * Min Line Length     :  11
    * Max Line Gap        :  22
    * Grey Thresholding   :  11
    * Hough Threshold     :  46
    * Contour Threshold   :  500
    
    * Raw Contour Areas        : [2762.0, 1751.0, 4845.5, 2999.0, 7708.5, 1574.0, 398.0])
    * Filtered Contour Areas   : [2762.0, 1751.0, 4845.5, 2999.0, 7708.5, 1574.0])
""")

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression,RANSACRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# ransac_img = cv2.imread("/home/hunter/devel/vision_playground/input-lines.png")
# nonzero_pxls = ransac_img.nonzero()
# nonzero_y = np.array(nonzero_pxls[0]).reshape(-1,1)
# nonzero_x = np.array(nonzero_pxls[1]).reshape(-1,1)


In [ ]:
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

nonzero_y = np.array(nonzero_pxls[0]).reshape(-1,1)
nonzero_x = np.array(nonzero_pxls[1]).reshape(-1,1)
ransac = RANSACRegressor()

estimators = [('OLS', LinearRegression()),
              ('Theil-Sen', TheilSenRegressor(random_state=42)),
              ('RANSAC', RANSACRegressor(random_state=42)),
              ('HuberRegressor', HuberRegressor())]
colors = {'OLS': 'turquoise', 'Theil-Sen': 'gold', 'RANSAC': 'lightgreen', 'HuberRegressor': 'black'}
linestyle = {'OLS': '-', 'Theil-Sen': '-.', 'RANSAC': '--', 'HuberRegressor': '--'}
lw = 3

x_plot = np.linspace(X.min(), X.max())
for title, this_X, this_y in [
        ('Modeling Errors Only', X, y),
        ('Corrupt X, Small Deviants', X_errors, y),
        ('Corrupt y, Small Deviants', X, y_errors),
        ('Corrupt X, Large Deviants', X_errors_large, y),
        ('Corrupt y, Large Deviants', X, y_errors_large)]:
    plt.figure(figsize=(5, 4))
    plt.plot(this_X[:, 0], this_y, 'b+')

    for name, estimator in estimators:
        model = make_pipeline(PolynomialFeatures(3), estimator)
        model.fit(this_X, this_y)
        mse = mean_squared_error(model.predict(X_test), y_test)
        y_plot = model.predict(x_plot[:, np.newaxis])
        plt.plot(x_plot, y_plot, color=colors[name], linestyle=linestyle[name],
                 linewidth=lw, label='%s: error = %.3f' % (name, mse))

    legend_title = 'Error of Mean\nAbsolute Deviation\nto Non-corrupt Data'
    legend = plt.legend(loc='upper right', frameon=False, title=legend_title,
                        prop=dict(size='x-small'))
    plt.xlim(-4, 10.2)
    plt.ylim(-2, 10.2)
    plt.title(title)
plt.show()

In [ ]:
x = np.arange(9.).reshape(3, 3)
print(x)
goodvalues = np.uint8(range(140,171))
print(goodvalues)
ix = np.isin(x, goodvalues)
a=np.where( x > 5 )
b = np.where( x < 7 )
c = np.bitwise_and(a,b)
print(a)
print(b)
print(c)


In [ ]:
nObs = 5

b = 2
d = np.ceil(nObs / float(b))
print(b, d)
plt.figure()
fig, axes = plt.subplots(nrows=2, ncols=int(d))
# print(axes.shape)
for ax in fig.axes:
    ax.plot([1,2,3,4], [1,4,9,16], 'ro')
    
plt.show()